In [69]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Percorsi
input_folder = 'FESI/Progetto/malaria/malaria/'  # Cartella delle immagini di input
output_folder = 'FESI/Progetto/malaria/output/'  # Cartella dove salvare le immagini elaborate
mask_folder = 'FESI/Progetto/malaria/masks/'  # Cartella dove salvare le maschere

# Crea le cartelle di output se non esistono
os.makedirs(output_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)

# Itera sui file nella cartella di input
for filename in os.listdir(input_folder):
    if filename.endswith('.png') or filename.endswith('.jpg'):  # Filtra i tipi di file immagine
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        mask_path = os.path.join(mask_folder, f"mask_{filename}")
        
        # Carica l'immagine
        image = cv2.imread(input_path)
        original = image.copy()

        # Passo 1: Riduzione del rumore con filtro mediano
        image = cv2.medianBlur(image, 5)

        # Passo 2: Riduzione del rumore con filtro bilaterale
        # Parametri: d = diametro, sigmaColor = intensità, sigmaSpace = vicinanza
        image = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

        # Converti l'immagine nello spazio colore HSV
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Creo delle maschere per filtrare lo sfondo
        non_zero_saturation = hsv[:, :, 1] > 0
        non_zero_hue = hsv[:, :, 0] > 0
        non_zero_value = hsv[:, :, 2] > 0

        # Filtra i pixel di saturazione > 0
        filtered_saturation = hsv[:, :, 1][non_zero_saturation]
        filtered_hue = hsv[:, :, 0][non_zero_hue]
        filtered_value = hsv[:, :, 2][non_zero_value]

        # Calcola gli istogrammi
        hist_h = cv2.calcHist([filtered_hue], [0], None, [180], [0, 180])
        hist_s_filtered = cv2.calcHist([filtered_saturation], [0], None, [256], [0, 256])
        hist_v = cv2.calcHist([filtered_value], [0], None, [256], [0, 256])

        # Calcola metriche per l'analisi della distribuzione
        filtered_saturation_mean = np.mean(filtered_saturation[filtered_saturation > 0])  # Saturazione media (escludendo sfondo)
        filtered_saturation_std = np.std(filtered_saturation[filtered_saturation > 0])    # Deviazione standard
        filtered_saturation_mode = np.argmax(hist_s_filtered)                   # Moda (valore più frequente)

        # Condizioni basate sull'analisi della distribuzione
        if filtered_saturation_std < 20:  # Distribuzione stretta
            saturation_threshold_max = min(255, filtered_saturation_mode + 20)
        else:  # Distribuzione ampia
            saturation_threshold_max = min(255, (int(filtered_saturation_mean + filtered_saturation_std)))
            
        saturation_threshold_max = saturation_threshold_max + 8 # Aggiungi un margine di sicurezza

        # Imposta i limiti HSV dinamici
        lower_purple = np.array([120, (saturation_threshold_max), 70])  # Limite inferiore (Hue, Saturation, Value)
        upper_purple = np.array([185, 255, 255])  # Limite superiore

        # Crea una maschera per isolare le aree viola
        mask = cv2.inRange(hsv, lower_purple, upper_purple)
        
        # Post-processing: Morfologia
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) # Apertura per rimuovere il rumore

        # 4. Operazioni morfologiche aggiuntive
        #dilated_mask = cv2.dilate(mask, kernel, iterations=2) 
        #final_mask = cv2.morphologyEx(dilated_mask, cv2.MORPH_CLOSE, kernel)         
                
        # Rimuovi le macchie viola dall'immagine
        image[mask > 0] = [255, 255, 255]          

        # Salva la maschera e l'immagine modificata
        cv2.imwrite(mask_path, mask)
        cv2.imwrite(output_path, image)

        # (Opzionale) Visualizza il progresso
        print(f"Elaborata immagine: {filename}")


Elaborata immagine: parasite (1).png
Elaborata immagine: parasite (10).png
Elaborata immagine: parasite (100).png
Elaborata immagine: parasite (101).png
Elaborata immagine: parasite (102).png
Elaborata immagine: parasite (103).png
Elaborata immagine: parasite (104).png
Elaborata immagine: parasite (105).png
Elaborata immagine: parasite (106).png
Elaborata immagine: parasite (107).png
Elaborata immagine: parasite (108).png
Elaborata immagine: parasite (109).png
Elaborata immagine: parasite (11).png
Elaborata immagine: parasite (110).png
Elaborata immagine: parasite (111).png
Elaborata immagine: parasite (112).png
Elaborata immagine: parasite (113).png
Elaborata immagine: parasite (114).png
Elaborata immagine: parasite (115).png
Elaborata immagine: parasite (116).png
Elaborata immagine: parasite (117).png
Elaborata immagine: parasite (118).png
Elaborata immagine: parasite (119).png
Elaborata immagine: parasite (12).png
Elaborata immagine: parasite (120).png
Elaborata immagine: parasite (

Visto che in alcune immagini (come la 13, inserisci il link) il valore di saturazione scelto causava un eccessiva rimozione della cellula. Il valore di saturazione scelto oscillava tra quelli più utilizzati (inserisci immagine con punto sull'istogramma), e non uno di quelli con frequenza minore, osservando questo istogramma a noi interessa un valore che stia più o meno tra i 125 (inserisci poi foto dell'istogramma e tutto).
Nelle immagini come la 54 dove l'istogramma di Hue è formato da 2 grandi picchi come valori, e la saturazione è invece un grande picco che si alza e poi si abbassa, il valore di saturazione sufficiente è s. Mettere prima i risultati "sbagliati" e poi quelli giusti. Vedi file .txt nell'altra cartella progetto. <br>
Sfrutto quindi un analisi dei due istogrammi (Hue e Saturation) per aggirare il problema 

### Visualizzazione istogramma 2d Per mettere in relazione Tonalità e saturazione (da vedere se sfruttabile)